In [1]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets
from sklearn.decomposition import PCA # ????
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.metrics import accuracy_score


# Perceptron

In [2]:
class Perceptron2(object):
    """Perceptron classifier.
    
    Parameters
    ------------
    eta: float 
        Learning rate (between 0.0 and 1.0)
    n_iter: int
        Number of epochs, i.e., passes over the training dataset.
        
    Attributes
    ------------
    w_: 1d-array
        Weights after fitting.
    errors_: list
        Number of misclassifications in every epoch.
    random_state : int
        The seed of the pseudo random number generator.
    """
    
    def __init__(self, eta=0.01, n_iter=10, random_state=1):
        self.eta = eta
        self.n_iter = n_iter
        self.random_state = random_state
    
    def fit(self, X, y):
        """Fit training data.

        Parameters
        ----------
        X : {array-like}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.
        y : array-like, shape = [n_samples]
            Target values.

        Returns
        -------
        self : object

        """
        rgen = np.random.RandomState(self.random_state)
        self.w_ = rgen.normal(loc=0.0, scale=0.01, size=1+X.shape[1])
        self.errors_ = []
        
        for _ in range(self.n_iter):
            errors = 0.0
            for xi, target in zip(X, y):
                update = self.eta * (self.predict(xi) - target)
                self.w_[1:] -= update * xi
                self.w_[0] -= update
                errors += int(update != 0.0)
            self.errors_.append(errors)
        return self
    
    def net_output(self, X):
        """Calculate net output"""
        return np.dot(X, self.w_[1:]) + self.w_[0]
    
    def predict(self, X):
        """Return class label after unit step"""
        return np.where(self.net_output(X) >= 0.0, 1, -1)
    

# Adaline

In [3]:
class Adaline(object):
    """ADAptive LInear NEuron classifier.

    Parameters
    ------------
    eta : float
        Learning rate (between 0.0 and 1.0)
    n_iter : int
        Passes over the training dataset.
    random_state : int
        The seed of the pseudo random number generator.

    Attributes
    -----------
    w_ : 1d-array
        Weights after fitting.
    errors_ : list
        Number of misclassifications in every epoch.

    """
    def __init__(self, eta=0.01, n_iter=50, random_state=1):
        self.eta = eta
        self.n_iter = n_iter
        self.random_state = random_state

    def fit(self, X, y):
        """ Fit training data.

        Parameters
        ----------
        X : {array-like}, shape = [n_samples, n_features]
            Training vectors, where n_samples is the number of samples and
            n_features is the number of features.
        y : array-like, shape = [n_samples]
            Target values.

        Returns
        -------
        self : object

        """
        rgen = np.random.RandomState(self.random_state)
        self.w_ = rgen.normal(loc=0.0, scale=0.01, size=1 + X.shape[1])
        self.cost_ = []

        for i in range(self.n_iter):
            output = self.activation(X)
            errors = (y - output)
            self.w_[1:] += self.eta * X.T.dot(errors)
            self.w_[0] += self.eta * errors.sum()
            cost = (errors**2).sum() / 2.0
            self.cost_.append(cost)
        return self

    def net_output(self, X):
        """Calculate net input"""
        return np.dot(X, self.w_[1:]) + self.w_[0]

    def activation(self, X):
        """Compute linear activation"""
        y = self.net_output(X)
        z = 1/(1 + np.exp(-y)) 
        return z

    def predict(self, X):
        """Return class label after unit step"""
        return np.where(self.activation(X) >= 0.0, 1, -1)

# Klasyfikacja dla 4 cech

In [4]:
def prepare_y(y, class_value):
    yy = []
    for x in y:
        if x == class_value:
            x = 1
        else:
            x = -1
        yy.append(x)
    y = yy
    return y

def get_data(n=4):
    iris = datasets.load_iris()
    X1 = iris.data[0:150, :n]
    yy1 = iris.target[0:150]

    X2 = iris.data[0:150, :n]
    yy2 = iris.target[0:150]

    X3 = iris.data[0:150, :n]
    yy3 = iris.target[0:150]
    
    y1 = prepare_y(yy1, 0)
    y2 = prepare_y(yy1, 1)
    y3 = prepare_y(yy1, 2)
    
    X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2, random_state=0)
    X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=0)
    X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size=0.2, random_state=0)
    
    sc = StandardScaler()
    sc.fit(X1)
    X1_train_std = sc.transform(X1_train)
    X1_test_std = sc.transform(X1_test)

    sc.fit(X2)
    X2_train_std = sc.transform(X2_train)
    X2_test_std = sc.transform(X2_test)

    sc.fit(X3)
    X3_train_std = sc.transform(X3_train)
    X3_test_std = sc.transform(X3_test)
    
#     sc = StandardScaler()
    sc.fit(X1)
    X1_std = sc.transform(X1)
    
    set1 = (X1_train_std, y1_train, X1_test_std, y1_test)
    set2 = (X2_train_std, y2_train, X2_test_std, y2_test)
    set3 = (X3_train_std, y3_train, X3_test_std, y3_test)
    
    return X1_std, set1, set2, set3, yy1

def test_total_performance(X, m1, m2, m3, yy1):
    good = 0
    a1 = np.arange(0,50)
    a2 = np.arange(50,100)
    a3 = np.arange(100,150)
    ac = np.arange(0,150)


    for a in ac:
        y1_pred0 = m1.net_output(X[a,:])
        y2_pred0 = m2.net_output(X[a,:])
        y3_pred0 = m3.net_output(X[a,:])
        if np.argmax([y1_pred0, y2_pred0, y3_pred0]) == yy1[a]:
            good = good+1

    return good

## Przygotowanie Danych

In [5]:
X1_std, set1, set2, set3, yy1 = get_data(n=4)
(X1_train_std, y1_train, X1_test_std, y1_test) = set1
(X2_train_std, y2_train, X2_test_std, y2_test) = set2
(X3_train_std, y3_train, X3_test_std, y3_test) = set3

## Testowanie perceptronu

In [6]:
ppn1 = Perceptron2(n_iter=100, eta=0.01, random_state=0)
ppn2 = Perceptron2(n_iter=100, eta=0.01, random_state=0)
ppn3 = Perceptron2(n_iter=100, eta=0.01, random_state=0)

ppn1.fit(X1_train_std, y1_train)
ppn2.fit(X2_train_std, y2_train)
ppn3.fit(X3_train_std, y3_train)

y1_pred = ppn1.predict(X1_test_std)
y2_pred = ppn2.predict(X2_test_std)
y3_pred = ppn3.predict(X3_test_std)

print('Misclassified samples: %d' % (y1_test != y1_pred).sum())
print('Accuracy: %.2f' % accuracy_score(y1_test, y1_pred))
print('Misclassified samples: %d' % (y2_test != y2_pred).sum())
print('Accuracy: %.2f' % accuracy_score(y2_test, y2_pred))
print('Misclassified samples: %d' % (y3_test != y3_pred).sum())
print('Accuracy: %.2f' % accuracy_score(y3_test, y3_pred))

test_total_performance(X1_std, ppn1, ppn2, ppn3, yy1)

Misclassified samples: 0
Accuracy: 1.00
Misclassified samples: 7
Accuracy: 0.77
Misclassified samples: 1
Accuracy: 0.97


141

## Testowanie Adaline

In [7]:
eta = 0.003
n_iter = 100

ada1 = Adaline(n_iter=n_iter, eta=eta, random_state=0)
ada2 = Adaline(n_iter=n_iter, eta=eta, random_state=0)
ada3 = Adaline(n_iter=n_iter, eta=eta, random_state=0)

ada1.fit(X1_train_std, y1_train)
ada2.fit(X2_train_std, y2_train)
ada3.fit(X3_train_std, y3_train)

y1_pred_a = ada1.predict(X1_test_std)
y2_pred_a = ada2.predict(X2_test_std)
y3_pred_a = ada3.predict(X3_test_std)

print('Misclassified samples: %d' % (y1_test != y1_pred_a).sum())
print('Accuracy: %.2f' % accuracy_score(y1_test, y1_pred_a))
print('Misclassified samples: %d' % (y2_test != y2_pred_a).sum())
print('Accuracy: %.2f' % accuracy_score(y2_test, y2_pred_a))
print('Misclassified samples: %d' % (y3_test != y3_pred_a).sum())
print('Accuracy: %.2f' % accuracy_score(y3_test, y3_pred_a))

test_total_performance(X1_std, ada1, ada2, ada3, yy1)

Misclassified samples: 19
Accuracy: 0.37
Misclassified samples: 17
Accuracy: 0.43
Misclassified samples: 24
Accuracy: 0.20


127